In [ ]:
import numpy as np
import dask.array as da
import zarr
from numcodecs import Blosc
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
from ome_zarr.writer import write_multiscales_metadata
import ome_zarr.format
src_directory = "D:/embryo_data/zarr_arrays/230601.152354.round14.001.Group1.A1.T001P01.ome.zarr"
save_directory = "D:/embryo_data/zarr_arrays/230601.152354.round14.001.Group1.A1.T001P01_cropped.ome.zarr"


In [ ]:
# load data
reader = Reader(parse_url(src_directory))
node = list(reader())[0]
src_data = node.data[0]

In [ ]:
src_data[1,0,0,-5,-5].compute()
# y=5:-5 x = 5:-5

# raveled_indices = da.argwhere(src_data > 2.2)
# unraveled_indices.unravel_index(raveled_indices, src_data.shape)

In [ ]:
cropped_src_data = src_data[:,:,:,5:-5,5:-5]
axes = node.metadata["axes"]
name = node.metadata["name"]
coord = node.metadata['coordinateTransformations'][0]

In [ ]:
# save data
data_group = zarr.open_group(save_directory, mode='w')
zarray_data = data_group.create_dataset(
    '0',
    shape=cropped_src_data.shape,
    chunks=(1,1,32,256,256),
    dtype=cropped_src_data.dtype,
    exact = True,
    compressor=Blosc(cname='zstd', clevel = 9, shuffle = 2),
    filters = [],
    dimension_separator = '/'
)
write_multiscales_metadata(
    group = data_group,
    datasets = [{
        "path": "0",
        "coordinateTransformations": coord
    }],
    fmt = ome_zarr.format.FormatV04(),
    name = name,
    axes = axes
)

cropped_src_data.store(zarray_data, lock=False, compute=True)